In [2]:

import pandas as pd
import os
from tqdm import tqdm
from multiprocessing import Pool 
import numpy as np


In [3]:

OUT_STEM = "/shared/3/projects/benlitterer/podcastData/intermediary/beforeFMonth/splitsFromParallel/fullTextSplit"
OUT_STEM = "/shared/3/projects/benlitterer/podcastData/intermediary/practice/transcriptFile"

In [7]:
df = pd.read_json("/shared/3/projects/benlitterer/podcastData/processed/mayJune/mayJuneMetadata.jsonl", orient="records", lines=True)

In [8]:
print(df.shape)

(1584643, 57)


In [9]:
def fileExists(fPath): 
    STEM_2 = "/shared/3/projects/benlitterer/podcastData/prosodyMerged/5_4_5_10"
    STEM_3 = "/shared/3/projects/benlitterer/podcastData/prosodyMerged/6_9_6_15"
    STEM_4 = "/shared/3/projects/benlitterer/podcastData/prosodyMerged/mayJuneRemaining"
    STEM_5 = "/shared/3/projects/benlitterer/podcastData/prosodyMerged/floydMonth"
    stemList = [STEM_1, STEM_2, STEM_3, STEM_4]
    for stem in stemList: 
        potPath = stem + fPath
        if os.path.exists(potPath): 
            return potPath 
    return "" 

df["fullPotPath"] = df["potentialOutPath"].apply(fileExists)
df = df[df["fullPotPath"] != ""]

In [14]:
for item in df.sample(10)["fullPotPath"]: 
    print(item)

/shared/3/projects/benlitterer/podcastData/prosodyMerged/mayJuneRemaining/anchor.fm/2t/httpsanchor.fms8a5f50cpodcastplay15760713https3A2F2Fd3ctxlq1ktw2nl.cloudfront.net2Fproduction2F20205262F85583180441002a6b91bfc214c1.mp3MERGED
/shared/3/projects/benlitterer/podcastData/prosodyMerged/beforeFMonth/awscdn.podcasts.com/9i/httpawscdn.podcasts.com9Einstein22cb9.mp3MERGED
/shared/3/projects/benlitterer/podcastData/prosodyMerged/mayJuneRemaining/traffic.libsyn.com/e9/httpstraffic.libsyn.comsecure9b3e7e4eff0f497c94e4827e5379224b33999.mp3destid3753240MERGED
/shared/3/projects/benlitterer/podcastData/prosodyMerged/5_4_5_10/anchor.fm/23/httpsanchor.fms13a72890podcastplay13518993https3A2F2Fd3ctxlq1ktw2nl.cloudfront.net2Fproduction2F2020492F716386664410026320d804277fb.mp3MERGED
/shared/3/projects/benlitterer/podcastData/prosodyMerged/6_9_6_15/traffic.libsyn.com/e_/httpstraffic.libsyn.comsecurehssbmThree_Levels_of_Prayers__Part_1.mp3destid2102864MERGED
/shared/3/projects/benlitterer/podcastData/pro

In [16]:
potPathDf = df[["fullPotPath"]]
dfList = np.array_split(potPathDf, 12)

/opt/anaconda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [27]:
def outputDataFile(i): 
    currDf = dfList[i]
    OUT_PATH = f"{OUT_STEM}{i}.tsv"
    with open(OUT_PATH, "w") as outFile: 
        if i == 0: 
            for path in tqdm(currDf["fullPotPath"]): 
                with open(path) as currFile: 
                    fullText =  "".join([row.split(",")[4] for row in currFile.readlines()])

                    #remove tabs and all types of newlines 
                    fullText = fullText.replace("\t", "")
                    fullText = " ".join(fullText.splitlines())
                    
                    outFile.write(f"{path}\t{fullText}\n")
        else: 
            for path in currDf["fullPotPath"]: 
                with open(path) as currFile: 
                    fullText =  "".join([row.split(",")[4] for row in currFile.readlines()])

                    #remove tabs and all types of newlines 
                    fullText = fullText.replace("\t", "")
                    fullText = " ".join(fullText.splitlines())
                    
                    outFile.write(f"{path}\t{fullText}\n")



In [28]:
pool = Pool(processes=12)
pool.map(outputDataFile, list(range(12)))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 343/343 [00:15<00:00, 21.67it/s]


[None, None, None, None, None, None, None, None, None, None, None, None]

In [26]:
df.shape

(4111, 58)